# SE-2200E Notebook 3: Data Transformation

Ningsong Shen

February 12, 2021

## Summary

We want to transform the data to make it easier to use, we want to select a subset of data to use, remove outliers, remove irrelevant features so that we can apply simple techniques. Professor Katchabaw suggested this method which would make testing and isolating any issues easier.

In [3]:
import pandas as pd
import numpy as np


In [4]:
df = pd.read_csv('../clean_data/prepared_data.csv')

C:\Users\nings\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df['trip_id'] = df['trip_id'].astype('category')
df.head()

,trip_id,start_date,route_id,stop_sequence,realtime_departure_time,vehicle_id,scheduled_departure_time
0,1346803,20201104,24,41,1604508849,3140.0,11:53:34
1,1346803,20201105,24,41,1604595120,3177.0,11:53:34
2,1346803,20201106,24,41,1604681520,3143.0,11:53:34
3,1346803,20201109,24,41,1604940720,3144.0,11:53:34
4,1346803,20201110,24,41,1605027120,3178.0,11:53:34


Let's choose some features: a single route id and stop sequence will identify a single stop.

## More Data Cleaning

To do a linear regression, we need to convert all the dates and times into a numeric value. We will use seconds from the departure time.

In [6]:
# Combine the start times with the existing dataframe.
df1 = pd.read_csv('../raw_data/schedule/stop_times.txt')
df2 = df1[df1["stop_sequence"] == 1][['trip_id', 'departure_time']].rename(columns={'departure_time': 'start_time'})
df2['trip_id'] = df2['trip_id'].astype('category')
df2.head()

,trip_id,start_time
0,1342560,6:13:00
57,1342561,7:33:00
114,1342562,6:53:00
175,1342563,8:02:00
236,1342564,9:02:00


In [6]:
combined_df = df.merge(df2, on="trip_id", how='inner')
combined_df['real_departure_time'] = pd.to_datetime(combined_df['realtime_departure_time'],unit='s').dt.tz_localize("GMT").dt.tz_convert('America/Toronto').dt.time
combined_df.head()

,trip_id,start_date,route_id,stop_sequence,realtime_departure_time,vehicle_id,scheduled_departure_time,start_time,real_departure_time
0,1346803,20201104,24,41,1604508849,3140.0,11:53:34,11:18:00,11:54:09
1,1346803,20201105,24,41,1604595120,3177.0,11:53:34,11:18:00,11:52:00
2,1346803,20201106,24,41,1604681520,3143.0,11:53:34,11:18:00,11:52:00
3,1346803,20201109,24,41,1604940720,3144.0,11:53:34,11:18:00,11:52:00
4,1346803,20201110,24,41,1605027120,3178.0,11:53:34,11:18:00,11:52:00


After the mess of converting times, we now had to find the number of seconds between the times so that a regression can be performed. Initially, I had no idea how to best model the dataset in a way that the regression can assist with prediction. Individual times and stops lacked sufficient data, times were not exactly numeric, and I wanted to be able to isolate different datasets to make testing easier. In a flash of inspiration, I found that the number of seconds elapsed from the terminal departure time was ideal. This would let me isolate by line, see patterns in journey times, and do better prediction with an abundance of data.

In [7]:
helper = np.vectorize(lambda x: x.total_seconds())
combined_df['real_departure_time'] = helper(pd.to_datetime(combined_df['real_departure_time'].astype(str)).dt.to_pydatetime() - pd.to_datetime(combined_df['start_time'].astype(str)).dt.to_pydatetime())

combined_df['scheduled_departure_time'] = helper(pd.to_datetime(combined_df['scheduled_departure_time'].astype(str)).dt.to_pydatetime() - pd.to_datetime(combined_df['start_time'].astype(str)).dt.to_pydatetime())

In [8]:
del combined_df["realtime_departure_time"]
del combined_df["start_time"]


In [9]:
combined_df.head()

,trip_id,start_date,route_id,stop_sequence,vehicle_id,scheduled_departure_time,real_departure_time
0,1346803,20201104,24,41,3140.0,2134.0,2169.0
1,1346803,20201105,24,41,3177.0,2134.0,2040.0
2,1346803,20201106,24,41,3143.0,2134.0,2040.0
3,1346803,20201109,24,41,3144.0,2134.0,2040.0
4,1346803,20201110,24,41,3178.0,2134.0,2040.0


## Selecting a Route and storing it in a csv

In [2]:
# Currently we try all data, a good representative sample is 1345770
# stop_route = combined_df.loc[(df['trip_id'] == 1345770)]
stop_route = combined_df

NameError: name 'combined_df' is not defined

In [1]:
stop_route.sample(20)

NameError: name 'stop_route' is not defined

In [12]:
stop_route.to_csv('../clean_data/sample_data.csv', index=False)